In [ ]:
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

In [ ]:
from imports import *
from networkx.readwrite.json_graph import node_link_graph
tqdm.pandas()
pd.set_option('display.max_colwidth', None)


In [ ]:
from models.sage import SAGE
from models.loss import DotCeLossWithOkvedDistances

In [ ]:
def get_pretrained_embedding_tensor(idx_to_okved: dict, path_to_embeddings_dict: str) -> torch.Tensor:
    """
    Создает тензор с эмбеддингами ОКВЭД

    Args:
        idx_to_okved (dict): маппинг номер кода ОКВЭД - код
        path_to_embeddings_dict: путь с словарю с эмбеддингами ОКВЭД
    Returns:
        torch.Tensor: тензор с эмбеддингами ОКВЭД
    """
    with open(path_to_embeddings_dict, 'rb') as fp:
        embeddings_dict = pickle.load(fp)
        embedding_size = len(next(iter(embeddings_dict.values())))
        embedding_tensor = torch.zeros(len(idx_to_okved), embedding_size)
    for idx, okved in idx_to_okved.items():
        if okved in embeddings_dict:
            embedding_tensor[idx] = torch.FloatTensor(embeddings_dict[okved])
    return embedding_tensor


In [ ]:
def create_dataset(dataset_dir: str, result_dir: str, etypes=None, **kwargs):
    """
    Обрабатывает все файлы из каталога и подгатавливает датасет

    Args:
        dataset_dir (str): путь к каталогу с pickle файлами с информацией о ГСК
        result_dir (str): путь для сохранения датасета
        etypes (list, optional): список типов узлов для добавления в датасет
    Returns:
        list: датасет для решения задачи классификации ребер как находящихся внутри ГСК или ведущих вовне ГСК

    """

    dataset = []
    for fname in tqdm(list(Path(dataset_dir).iterdir())):
        with open(fname, 'rb') as fp:
            data = pickle.load(fp)
        node_membership = {}

        data['nodes'] = [node for node in data['nodes'] if node['okved code'] != 'unknown']

        # атрибуты узлов - номер кода ОКВЭД, финансовые показатели, ID компании
        # оставляем только членов ГСК
        for node in data['nodes']:
            node['okved_code'] = okved_to_idx[node['okved_code']]
            node['profit'] = np.array([node[k] for k in ['p21103', 'p21104']])
            node['company_id'] = node['id']
            node['gc_root'] = data['graph']['root']
            node_membership[node['id']] = node['входит_в_ГСК']

        # убираем ребра для удаленных узлов
        data['links'] = [link for link in data['links'] if (link['source'] in node_membership and
                                                            link['target'] in node_membership)]

        for edge in data['links']:
            edge['gc_root'] = data['graph']['root']
            # 1 если оба узла входят в ГСК, 0 - если хотя бы один не входит
            edge['label'] = int(node_membership[edge['source']] and node_membership[edge['target']])

        # фильтруем типы связей
        if etypes is not None:
            data['links'] = [edge for edge in data['links'] if edge['key'] in etypes]
        if not data['links'] or not data['nodes']:
            continue

        H = node_link_graph(data, directed=True, multigraph=False)
        # убираем изолированные узлы, появившиеся в результате удаления других узлов или связей
        H.remove_nodes_from(list(nx.isolates(H)))
        if 'root' not in H.graph and 'ГСK_root' in data:
            H.graph['root'] = data['ГСК_root']
        g = dgl.from_networkx(H,
                              node_attrs=['profit', 'okved_code', 'company_id', 'входит_в_ГСК', 'gc_root'],
                              edge_attrs=['label'])

        in_deg = g.in_degrees().view(-1, 1)
        out_deg = g.out_degrees().view(-1, 1)
        raw_profit = g.ndata['profit']

        # вместо абсолютных значений берем знак raw_profit
        profit = torch.where(raw_profit < 0, 0,
                             torch.where(raw_profit == 0, 1, 2))
        g.ndata['feats'] = torch.cat([profit.view(-1, 2),
                                      in_deg,
                                      out_deg], dim=1)

        # 80% на обучение
        g.edata['train_mask'] = torch.zeros(g.num_edges(), dtype=torch.bool).bernoulli(0.8)
        g.gc_root = data['graph']['root']
        dataset.append(g)

    # фильтруем слишком маленькие или слишком большие графы
    dataset = [g for g in dataset if 5 <= g.num_nodes() <= 50]
    random.shuffle(dataset)

    with open(result_dir, 'wb') as fp:
        pickle.dump(dataset, fp)
    return dataset

In [ ]:
def train(dataset: list,
          company_embeddings_model_path: str,
          embedding_tensor: torch.Tensor,
          n_hidden: int = 32, n_out: int = 8,
          max_no_improvements: int = 5,
          print_each: int = 5,
          n_epochs: int = 10):
    """
    Обучает модель для решения задачи классификации ребер как находящихся внутри ГСК или ведущих вовне ГСК
    Args:
        dataset (list[dgl.DGLGraph]): датасет для обучения модели
        company_embeddings_model_path (str): путь для сохранения
        embedding_tensor: torch.Tensor,
        n_hidden (int): размерность скрытых слоев модели
        n_out (int): размерность выходного слоя модели
        print_each (int): шаг для отображения текущих результатов внутри эпохи
        max_no_improvements (int): максимальное кол-во эпох без улучшения качества
        n_epochs
    Returns:
        SAGE: обученная модель для классификации ребер как находящихся внутри ГСК или ведущих вовне ГСК
    """
    n_features = dataset[0].ndata['feats'].shape[1] + embedding_tensor.shape[1]
    model = SAGE(n_features, n_hidden,
                 n_out, n_layers=2,
                 activation=F.relu, dropout=0.15,
                 freeze=True,
                 embedding_tensor=embedding_tensor)

    optimizer = optim.Adam(model.parameters(), lr=0.005)

    all_labels = torch.cat([g.edata['label'] for g in dataset])
    neg_counts, pos_counts = all_labels.bincount()
    criterion = DotCeLossWith0kvedDistances(model.embeddings, pos_weight=pos_counts / neg_counts, okved_impact=0.5)
    best_acc = 0
    no_improvements = 0

    dataloader = dgl.dataloading.GraphDataLoader(dataset, batch_size=256, shuffle=True, drop_last=False)

    for epoch in range(n_epochs):
        train_correct, test_correct, train_total, test_total = 0, 0, 0, 0
        epoch_test_preds = []
        epoch_test_labels = []
        for g in tqdm(dataloader):
            node_features = g.ndata['feats'].float()
            okveds = g.ndata['okved_code'].long()
            edge_label = g.edata['label']
            train_mask = g.edata['train_mask']
            preds = model(g, node_features, okveds)
            loss, score, labels = criterion(g, preds, okveds, edge_label, train_mask)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            train_predictions = (score.sigmoid() > 0.5)
            train_correct += (train_predictions == labels).sum()
            train_total += len(labels)
            _, test_score, test_labels = criterion(g, preds, okveds, edge_label, ~train_mask)
            test_predictions = (test_score.sigmoid() > 0.5)
            test_correct += (test_predictions == test_labels).sum()
            test_total += len(test_labels)
            epoch_test_preds.extend(test_predictions.tolist())
            epoch_test_labels.extend(test_labels.tolist())

        test_acc = test_correct / test_total
        train_acc = train_correct / train_total

        if epoch % print_each == 0 or epoch == n_epochs - 1:
            print(
                f'{epoch=:05d} | loss={loss.item():.4f} | train_acc={train_acc.item():.4f} | test_acc={test_acc.item():.4f}')
        if test_acc - best_acc >= 1e-3:
            print(f'New best acc: {test_acc}!')
            best_acc = test_acc
            no_improvements = 0
            torch.save(model, company_embeddings_model_path)
        else:
            no_improvements += 1
        if no_improvements >= max_no_improvements:
            print(f'No improvements in {max_no_improvements} epochs')
            print(f'Best acc = {best_acc}')
            break
    model = torch.load(company_embeddings_model_path)
    return model

In [ ]:
def get_company_graph_with_embeddings(dataset: list, model: SAGE) -> dgl.DGLGraph:
    """
    Строит граф, содержащий данные о всех компаниях датасета

    Args:
        dataset (list[dgl.DGLGraph]): датасет для обучения модели
        model: обученная модель для классификации ребер как находящихся внутри ГСК или ведущих вовне ГСК
    Returns:
        граф, содержащий данные о всех компаниях датасета с эмбеддингами узлов на атрибутах
    """
    model.eval()
    batch = dgl.batch(dataset)
    batch_features = batch.ndata['feats'].float()
    batch_okveds = batch.ndata['okved_code'].long()
    all_embeddings = model(batch, batch__features, batch_okveds).detach()
    batch.ndata['embeddings'] = all_embeddings
    return batch


In [ ]:
# загружаем конфигурационный файл
CONFIG = yaml.safe_load(open("CONFIG.yaml', encoding='utf8'))
# получаем код ОКВЭД для каждой компании из датасета
okved_parts = ['okved_class_', 'okved_subclass', 'okved_group', 'okved_subgroup', 'okved_type_']
okved_data = pd.read_csv(CONFIG['paths']['okved_data_save'],
                         index_col=0,
                         dtype={c: str for c in okved_parts})

# вспомогательные словари для маппинга ОКВЭДов в целые числа
idx_to_okved = okved_data['okved'].to_dict()
okved_to_idx = {v: k for k, v in idx_to_okved.items()}
section_to_idx = {s: idx for idx, s in enumerate(okved_data['раздел'].unique())}
okved_to_section = okved_data[['okved', 'раздел']].set_index('okved')['раздел'].map(section_to_idx).to_dict()

# создаем тензор с эмбеддингами
embedding_tensor = get_pretrained_embedding_tensor(idx_to_okved, CONFIG['paths']['okved_embeddings'])
# загружаем датасет
inout_path = CONFIG['paths']['inout_dataset']
gc_path = CONFIG['paths']['gc_augmented_save']
if isfile(inout_path):
    with
open(inout_path, 'rb') as fp:
dataset = pickle.load(fp)
else:
dataset = create_dataset(gc_path, inout_path)
# обучаем модель
model = train(dataset,
              company_embeddings_model_path=CONFIG['paths']['company_embeddings_model'],
              embedding_tensor=embedding_tensor,
              n_hidden=32,
              n_out=8,
              max_no_improvements=5,
              print_each=5,
              n_epochs=15)
# собираем один большой граф, где есть все компании и их эмбеддинги
batch = get_company_graph_with_embeddings(dataset, model)


In [ ]:
def print_gc_and_companies(dataset: list, lower: int, upper: int):
    """
    Выводит на экран крупные ГСК и компании, которые в них входят
    """
    gc_graph = {g.gc_root: g for g in dataset}
    large_gcs = [g.gc_root for g in dataset if g.ndata['входит_в_ГСК'].bool().sum() >= 15][lower:upper]
    for gc_id in large_gcs:
        g = gc_graph[gc_id]
        companies = g.ndata['company_id'][g.ndata['входит_в_ГСК'].bool()].tolist()
        print(f'ID ГСК: {gc_id} Компании: {companies[:10]}')

In [2]:
def print_gc_ranked_table(okved_data: pd.DataFrame,
                          gc_id: int,
                          node_id: int,
                          model: SAGE,
                          max_k: int = 20) -> None:
    """
    Строит и выводит на экран таблицу близости компаний внутри ГСК gcid относительно целевой компании node_id
    Args:
        okved_data (pd.DataFrame): таблица с информацией об ОКВЭД
        gc_id (int): ID целевой ГСК
        node_id (int): ID целевого узла
        model (SAGE): обученная модель для классификации ребер как находящихся внутри ГСК или ведущих вовне ГСК
        max_k (int, optional): максимальное кол-во строк для вывода
        
    """
    
def print_same_okved_ranked_table(batch: dgl.DGLGraph,
                                  okved_data: pd.DataFrame,
                                  idx_to_okved: dict,
                                  gc_id: int,
                                  node_id: int,
                                  model: SAGE,
                                  max_k: int = 20):
    """
    Строит и выводит на экран таблицу близости компаний, имеющих одинаковый код ОКВЭД,
    относительно целевой компании node id
    Args:
        batch (dgl.DGLGraph): граф, содержащий данные о всех компаниях датасета с эмбеддингами узлов на атрибутах
        okved_data (pd.DataFrame): таблица с информацией об ОКВЭД
        idx_to_okved (dict): маппинг номер кода ОКВЭД - код
        gc_id (int): ID целевой ГСК
        node_id (int): ID целевого узла
        model (SAGE): обученная модель для классификации ребер как находящихся внутри ГСК или ведущих вовне ГСК
        max_k (int, optional): максимальное кол-во строк для вывода
    """

In [ ]:
def print_gc_ranked_table(okved_data: pd.DataFrame,
                          gc_id: int,
                          node_id: int,
                          model: SAGE,
                          max_k: int = 20) -> None:
    """
    Строит и выводит на экран таблицу близости компаний внутри ГСК gc_id относительно целевой компании node_id

    Args:
        okved_data (pd.DataFrame): таблица с информацией об ОКВЭД
        gc_id (int): ID целевой ГСК
        node_id (int): ID целевого узла
        model (SAGE): обученная модель для классификации ребер как находящихся внутри ГСК или ведущих вовне ГСК
        max_k (int, optional): максимальное кол-во строк для вывода
    """
    gc_graph = {g.gc_root: g for g in dataset}
    assert gc_id in gc_graph
    g = gc_graph[gc_id]
    with g.local_scope():
        # получаем эмбеддинги для всех узлов
        gc_embeddings = model(g, g.ndata['feats'].float(), g.ndata['okved_code'].long())
        g.ndata['embeddings'] = gc_embeddings
        # берем подграф на компаниях, входящих в ГСК
        in_gc_mask = g.ndata['входит_в_ГСК'].bool()
        h = g.subgraph(in_gc_mask)
        # считаем расстояния между каждой парой узлов (на основе эмбеддингов)
        gc_embeddings = h.ndata['embeddings'].detach()
        pw_dists = torch.Tensor(pairwise_distances(gc_embeddings, metric='euclidean'))
        # node_graph_idx - номер целевой компании в подграфе
        node_graph_idx = (h.ndata['company_id'] == node_id).nonzero().item()
        g.ndata.pop('embeddings')
        # берем ближайшие компании
        dist_from_source = pw_dists[node_graph_idx]
        k = min(max_k, len(pw_dists))
        values, indices = dist_from_source.topk(k, largest=False)
        comp_id = h.ndata['company_id'][indices]
        # … их коды ОКВЭД
        okved_id = [idx_to_okved[i] for i in h.ndata['okved_code'][indices].tolist()]
        okved_name = okved_data.set_index('okved')['name'][okved_id]
        print("Целевой узел: ", node_id, "ГСК: ", gc_id)
        df = pd.DataFrame({'Ранг от целевого узла': np.arange(k),
                           'Расстояние по эмбеддингам': values.tolist(),
                           'ID компании': comp_id,
                           'Код ОКВЭД': okved_id,
                           'Код ОКВЭД с расшифровкой': okved_name,
                           'Входит в ГСК': h.ndata['входит_в_ГСК'][indices].tolist()})
        df['Ранг от целевого узла'] = np.arange(len(df))
        display(df)

In [ ]:
def print_same_okved_ranked_table(batch: dgl.DGLGraph,
                                  okved_data: pd.DataFrame,
                                  idx_to_okved: dict,
                                  gc_id: int,
                                  node_id: int,
                                  model: SAGE,
                                  max_k: int = 20):
    """
    Строит и выводит на экран таблицу близости компаний, имеющих одинаковый код ОКВЭД, 
    относительно целевой компании node id

    Args:
        batch (dgl.DGLGraph): граф, содержащий данные о всех компаниях датасета с эмбеддингами узлов на атрибутах
        okved_data (pd.DataFrame): таблица с информацией об ОКВЭД
        idx_to_okved (dict): маппинг номер кода ОКВЭД - код
        gc_id (int): ID целевой ГСК
        node_id (int): ID целевого узла
        model (SAGE): обученная модель для классификации ребер как находящихся внутри ГСК или ведущих вовне ГСК
        max_k (int, optional): максимальное кол-во строк для вывода
    """
    gc_graph = {g.gc_root: g for g in dataset}
    assert gc_id in gc_graph
    target_g = gc_graph[gc_id]
    okveds = target_g.ndata['okved_code'].long()
    # находим номер ОКВЭД целевой компании
    source_okved_id = okveds[(target_g.ndata['company_id'] == node_id).nonzero().item()]
    # отбираем все компании с тем же кодом
    same_okved_mask = batch.ndata['okved_code'] == source_okved_id
    companies = batch.ndata['company_id'][same_okved_mask].tolist()
    gc_ids = batch.ndata['gc_root'][same_okved_mask].tolist()
    embs = {}
    # итерируемся по компаниям с тем же ОКВЭД
    for i, c in zip(gc_ids, companies):
        # берем ГСК очередной компании и получаем эмбеддинги для всех узлов
        g = gc_graph[i]
        gc_embeddings = model(g, g.ndata['feats'].float(), g.ndata['okved_code'].long())
        # забираем эмбеддинг интересующего нас узла и кладем в словарь
        node_graph_idx = (g.ndata['company_id'] == c).nonzero().item()
        embs[(i, c)] = gc_embeddings[node_graph_idx]

    res = []
    for (gid, cid), emb in embs.items():
        res.append({'ID ГСК': gid,
                    'ID Компании': cid,
                    'Расстояние от целевой вершины': torch.dist(embs[(gc_id, node_id)], embs[(gid, cid)]).item()
                    })

    # ID ГСК читать как: узел из ГСК или около ГСК
    print("Целевой узел: ", node_id, ' ГСК: ', gc_id, 'код ОКВЭД: ', idx_to_okved[source_okved_id.item()])
    res = pd.DataFrame(res).sort_values("Расстояние от целевой вершины").reset_index(drop=True).head(max_k)
    display(res)
    

In [ ]:
print_gc_and_companies(dataset, 0, 10)

In [ ]:
print_gc_ranked_table(okved_data, gc_id=4755870, node_id=7887927, model=model, max_k=10)

In [ ]:
print_same_okved_ranked_table(batch, okved_data, idx_to_okved, gc_id=4755870, node_id=7887927, model=model, max_k=10)